### Day 19

In [79]:
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import defaultdict, Counter

In [2]:
day_i = 19

In [3]:
%run start_day.py $day_i

Initializing day 19


In [4]:
cd /home/vincent/Documents/AdventOfCode/2021

/home/vincent/Documents/AdventOfCode/2021


In [5]:
PATH = f"day{day_i}/input{day_i}"

In [6]:
!wc -l $PATH

716 day19/input19


In [7]:
!head $PATH

--- scanner 0 ---
-750,-798,564
651,-651,-450
545,505,-850
-869,-573,-721
-709,660,754
391,436,558
-748,-884,535
585,-474,445
618,-518,-448


In [18]:
!tail $PATH

821,-619,642
818,-543,610
343,700,-535
-666,-346,697
-684,425,-810
-709,-328,645
-699,-348,564
555,-659,-821
-624,-843,-636
-685,-867,-665


In [12]:
def parse_input(inp):
    scanners = []
    points = []
    assert "scanner 0" in inp[0]
    for x in inp[1:]:
        if "scanner" in x:
            scanners.append(points)
            points = []
        elif len(x) > 0:
            points.append(tuple(list(map(int, x.split(',')))))
    scanners.append(points)
    return scanners

In [13]:
# real input
with open(PATH, 'r') as f:
    inputs = parse_input([x.strip() for x in f.readlines()])


In [15]:
len(inputs)

26

In [19]:
# test input
test_str1 = """--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14"""

inputs_1 = parse_input(test_str1.split('\n'))

In [112]:
def rotations(n=3):
    assert n == 3
    # first dimension
    first_dim = []
    for i in range(n):
        first_dim.append((i, 1))
        first_dim.append((i, -1))
    # second dimension
    second = []
    for f, s in first_dim:
        for j in range(n):
            if j != f:
                second.append([(f, s), (j, 1)])
                second.append([(f, s), (j, -1)])
    # third
    rotations = []
    for match in second:
        rot = np.zeros((n, n))
        rot[0, match[0][0]] = match[0][1]
        rot[1, match[1][0]] = match[1][1]
        missing = 3 - match[0][0] - match[1][0]
        rot[2, missing] = 1
        if np.linalg.det(rot) == -1: rot[2, missing] = -1
        rotations.append(rot.astype(int))
    return rotations

In [211]:
ROTATIONS = rotations()

class Scanner():
    def __init__(self, l):
        self.points = l
        self.n_points = len(l)
        self.compute_distances()
        
    def compute_distances(self):
        self.distances = np.zeros((self.n_points, self.n_points))
        for i in range(self.n_points):
            for j in range(i+1, self.n_points):
                d = np.linalg.norm(np.array(self.points[i]) - np.array(self.points[j]), ord=2)
                self.distances[i, j] = d
                self.distances[j, i] = d
                
    def compare(self, o):
        """
        returns:
         - whether the two scanners overlap
         - if they do, return the matching
        """
        matching = {}
        ours, theirs = set(range(self.n_points)), set(range(o.n_points))
        for i in range(self.n_points):
            distances = set(self.distances[i, :])
            found, index = False, None
            for j in theirs:
                if not found:
                    their_distances = set(o.distances[j, :])
                    if len(distances & their_distances) >= 12:
                        found = True
                        index = j
            if found:
                matching[i] = index
                theirs.remove(index)
        if len(matching) >= 12: return True, matching
        else: return False, None
    
    def merge(self, o):
        """
        Assumes that the scanners overlap.
        Merges the points from o into self.
        """
        _, matching = self.compare(o)
        points_to_match = np.zeros((len(matching), 3))
        points_to_match2 = np.zeros((len(matching), 3))
        for i, k in enumerate(matching):
            points_to_match[i, :] = np.array(self.points[k])
            points_to_match2[i, :] = np.array(o.points[matching[k]])
        # get 12 differences
        vectors, vectors2 = np.zeros((len(matching), 3)), np.zeros((len(matching), 3))
        vectors[0, :] = points_to_match[2, :] - points_to_match[1, :]
        vectors2[0, :] = points_to_match2[2, :] - points_to_match2[1, :]
        for i in range(1, len(matching)):
            vectors[i, :] = points_to_match[i, :] - points_to_match[0, :]
            vectors2[i, :] = points_to_match2[i, :] - points_to_match2[0, :]
        # find rotation
        found_rotation, rotation = False, None
        for ROT in ROTATIONS:
            if np.allclose(vectors2 @ ROT, vectors):
                assert not found_rotation
                found_rotation = True
                rotation = ROT
        assert found_rotation
        offset = (points_to_match2 @ rotation - points_to_match)[0, :]
        # convert points from o
        converted_points = []
        for p in o.points:
            new_point = np.array(p) @ rotation - offset
            new_point = tuple(new_point.astype(int).tolist())
            converted_points.append(new_point)
        all_points = set(self.points)
        for p in converted_points:
            if p not in all_points:
                all_points.add(p)
        return Scanner(list(all_points)), offset

In [194]:
scanner0 = Scanner(inputs_1[0])
scanner1 = Scanner(inputs_1[1])
scanner4 = Scanner(inputs_1[4])
scanner0.compare(scanner1)

(True,
 {0: 3,
  1: 8,
  3: 12,
  4: 1,
  5: 24,
  6: 18,
  7: 10,
  9: 0,
  12: 2,
  14: 5,
  19: 15,
  24: 19})

In [195]:
scanner0.merge(scanner1)

38


In [187]:
scanner1.compare(scanner4)

(True,
 {2: 4,
  6: 11,
  8: 24,
  13: 1,
  15: 18,
  16: 15,
  18: 17,
  19: 5,
  21: 13,
  22: 12,
  23: 16,
  24: 3})

In [226]:
def solve1(inp):
    n = len(inp)
    scanners = [Scanner(x) for x in inp[1:]]
    current = Scanner(inp[0])
    to_merge = set(range(len(scanners)))
    while len(to_merge) > 0:
        for i in to_merge:
            overlap, _ = current.compare(scanners[i])
            if overlap:
                current, _ = current.merge(scanners[i])
                to_merge.remove(i)
                print(f"Merged with {i+1} - new count = {current.n_points}")
                break
    return current.n_points

def solve2(inp):
    n = len(inp)
    scanners = [Scanner(x) for x in inp[1:]]
    # scanner_locations
    scanner_locs = np.zeros((n, 3)) # scanner 0's location is automatically good
    # merging scanners
    current = Scanner(inp[0])
    to_merge = set(range(len(scanners)))
    while len(to_merge) > 0:
        for i in to_merge:
            overlap, _ = current.compare(scanners[i])
            if overlap:
                current, offset = current.merge(scanners[i])
                scanner_locs[i+1, :] = offset
                to_merge.remove(i)
#                 print(f"Merged with {i+1} - new count = {current.n_points}")
                break
    # compute distances
    distances = []
    for i in range(n):
        for j in range(i):
            distances.append(np.sum(np.abs(scanner_locs[i, :] - scanner_locs[j, :])))
    return max(distances)

In [213]:
solve1(inputs_1)

Merged with 1 - new count = 38
Merged with 3 - new count = 51
Merged with 4 - new count = 65
Merged with 2 - new count = 79


79

In [200]:
solve1(inputs)

40
Merged with 14 - new count = 40
53
Merged with 7 - new count = 53
67
Merged with 10 - new count = 67
81
Merged with 16 - new count = 81
94
Merged with 17 - new count = 94
102
Merged with 19 - new count = 102
115
Merged with 5 - new count = 115
129
Merged with 2 - new count = 129
143
Merged with 12 - new count = 143
157
Merged with 4 - new count = 157
170
Merged with 6 - new count = 170
183
Merged with 8 - new count = 183
197
Merged with 20 - new count = 197
210
Merged with 13 - new count = 210
217
Merged with 21 - new count = 217
222
Merged with 22 - new count = 222
236
Merged with 18 - new count = 236
249
Merged with 15 - new count = 249
263
Merged with 9 - new count = 263
270
Merged with 11 - new count = 270
278
Merged with 23 - new count = 278
292
Merged with 1 - new count = 292
306
Merged with 3 - new count = 306
319
Merged with 24 - new count = 319
320
Merged with 25 - new count = 320


320

In [227]:
solve2(inputs_1)

3621.0

In [228]:
solve2(inputs)

9655.0